### Imports and Other Calls

In [3]:
!git clone https://github.com/MikhaelBelmiro/Thousand-Sunny-IFEST.git

Cloning into 'Thousand-Sunny-IFEST'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 42 (delta 7), reused 18 (delta 2), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [4]:
#Utilities
import os
import pandas as pd
import numpy as np
from datetime import datetime as dt

#Visualization
import matplotlib as mpl
from plotly import express as px
from plotly import graph_objects as go
from matplotlib import pyplot as plt

#Machine Learning and Shit
from sklearn.preprocessing import MinMaxScaler

### Function Definition

In [23]:
def read_prepare_saham(file_path):
  df = pd.read_csv(file_path)[['Date', 'Adj Close', 'Volume']]
  df['Date'] = pd.to_datetime(df['Date'])
  df = df.set_index('Date').resample('D').last().fillna(method='bfill').reset_index()
  df = df.rename(columns={'Date':'date', 'Adj Close':'adj close', 'Volume':'volume'})
  return df

def date_formatter_berita(df):
  df['date'] = df['date'].apply(lambda x: dt.strptime(x, '%d %B %Y %H:%M').date())
  df['date'] = pd.to_datetime(df['date'])
  return df

def target_creation(df, diff_days:list):
  for i in diff_days:
    df[f'diff{i}_adj_close'] = df.diff(periods=i)['adj close'].shift(-i)
    df[f'target_{i}day'] = df[f'diff{i}_adj_close'].apply(lambda x: 1 if x > 0 else 0)
  return df

### Target Definition

In [44]:
antm_news = date_formatter(pd.read_csv('/content/Thousand-Sunny-IFEST/berita emiten/ANTM_news_title.csv').dropna())
giaa_news = date_formatter(pd.read_csv('/content/Thousand-Sunny-IFEST/berita emiten/GIAA_news_title.csv').dropna())
kaef_news = date_formatter(pd.read_csv('/content/Thousand-Sunny-IFEST/berita emiten/KAEF_news_title.csv').dropna())
unvr_news = date_formatter(pd.read_csv('/content/Thousand-Sunny-IFEST/berita emiten/UNVR_news_title.csv').dropna())
wskt_news = date_formatter(pd.read_csv('/content/Thousand-Sunny-IFEST/berita emiten/WSKT_news_title.csv').dropna())

In [31]:
giaa_saham = target_creation(read_prepare_saham('/content/Thousand-Sunny-IFEST/stock emiten/GIAA.JK.csv'), [i for i in range(1,3)])
antm_saham = target_creation(read_prepare_saham('/content/Thousand-Sunny-IFEST/stock emiten/ANTM.JK.csv'), [i for i in range(1,15)])
kaef_saham = target_creation(read_prepare_saham('/content/Thousand-Sunny-IFEST/stock emiten/KAEF.JK.csv'), [i for i in range(1,15)])
unvr_saham = target_creation(read_prepare_saham('/content/Thousand-Sunny-IFEST/stock emiten/UNVR.JK.csv'), [i for i in range(1,15)])
wskt_saham = target_creation(read_prepare_saham('/content/Thousand-Sunny-IFEST/stock emiten/WSKT.JK.csv'), [i for i in range(1,15)])

In [45]:
giaa = pd.merge(giaa_news, giaa_saham, on='date', how='inner').sort_values('date')
antm = pd.merge(antm_news, antm_saham, on='date', how='inner').sort_values('date')
kaef = pd.merge(kaef_news, kaef_saham, on='date', how='inner').sort_values('date')
unvr = pd.merge(unvr_news, unvr_saham, on='date', how='inner').sort_values('date')
wskt = pd.merge(wskt_news, wskt_saham, on='date', how='inner').sort_values('date')

In [65]:
!git push main main

fatal: could not read Username for 'https://github.com': No such device or address


### Pretraining Dataset

In [ ]:
invest = pd.read_csv('/content/Thousand-Sunny-IFEST/dataset judul berita/investment_titles.csv').drop(columns=['Unnamed: 0'])
market = pd.read_csv('/content/Thousand-Sunny-IFEST/dataset judul berita/market_titles.csv').drop(columns=['Unnamed: 0'])
finetune_df = pd.concat([invest, market])
finetune_df.columns = ['text']
finetune_df

,text
0,"Banyak Rumor Sesat, Catat Hal Ini Sebelum Inve..."
1,"Live Now! Euforia Bukalapak Lewat, Cek Valuasi..."
2,Jangan Tertipu! Investasi Bodong Mulai Pasang ...
3,Jurus Amankan Portofolio di Tengah Isu Tapering
4,"Bocoran nih Bunda, Besok Harga Emas di Pegadai..."
...,...
41294,"Mau Bangun LRT Rp 400 T, Saham Ratu Prabu Teta..."
41295,"Rupiah Menguat Kembali, Dolar AS Jatuh ke Rp 1..."
41296,"Kompak dengan Bursa Asia Lainnya, IHSG Dibuka ..."
41297,Kembalinya Dana Asing
